# 多表查询

## 表与表之间的关系
- 一对一：用户表和身份信息表，用户表是主表
    - 男人表，女人表
        - create table man(mid int primary key auto_increment, mname varchar(32) wid int unique);
        - create table woman(wid int primary key auto_increment, manme varchar(32) )
- 一对多：最常见的表关系，用户表和订单表
    - 员工表和部门表（见lesson_3） 
- 多对多：学生表和课程表，通常情况都是将多对多的关系拆分成一对多或多对一的关系
    - 至少需要三张表
        - create table students(sid int primary key auto_increment, sname varchar(32));
        - insert students (sname) vlaues ('大拿');
        - insert students (sname) vlaues ('二拿');
        - insert students (sname) vlaues ('三拿');
        - create table course(cid int primary key auto_increment, cname varchar(32));
        - insert course (cname) vlaues ('语文');
        - insert course (cname) vlaues ('数学');
        - insert course (cname) vlaues ('英语');
        - insert course (cname) vlaues ('化学');
        - create table s_c(cid int, sid int ); 
        - insert s_c (sid, cid) values (1, 1);
        - insert s_c (sid, cid) values (1, 2);
        - insert s_c (sid, cid) values (1, 3);
        - insert s_c (sid, cid) values (1, 4);
        - insert s_c (sid, cid) values (2, 2);
        - insert s_c (sid, cid) values (2, 4);
        - insert s_c (sid, cid) values (3, 1);
        - insert s_c (sid, cid) values (3, 3);

## 为什么要使用多张表
- 避免出现大量的数据的冗余；
- 并不是表拆的越多就越好，根据实际情况进行拆分

## 多表查询的概念
- 同时查询多张表

## 多表查询的分类
### 合并查询
- union，union all
- 合并结果集，就是把两个select语句的查询结果合并到一起。(相当于并集)
- 合并的两个结果、列数、列的顺序和类型都需要一致
- select * from emp union select * from dept;

### 连接查询
- 员工表：create table emp(empno int primary key auto_increment, ename varchar(32), job varchar(32), mgr int, hiredate date, sal double, comm double, deptno int );
- 部门表：create table dept(deptno int primary key auto_increment, dname varchar(32), loc varchar(32) );
- 内连接：左右两个表里符合条件的数据
    - inner join 是比较运算符，只返回符合条件的行
    - select * from emp inner join dept on emp.deptno=dept.deptno;  -- 后面on可以避免笛卡尔集的冗余情况
    - select * from emp e, dept d where e.deptno=d.deptno;  -- where也是条件
    - select * from emp e join dept d where e.deptno=d.deptno;
- 外连接：
    - 左外连结：左表行的全部，右表没有则用null ！！！(left outer join...on | left join...on )
        - select * from emp e left outer join dept d on e.deptno=d.deptno; 
    - 右外连接：右表行的全部，左表没有则用null ！！！(right outer join...on | right join...on )
        - select * from emp e right outer join dept d on e.deptno=d.deptno; 
- 自连接：
    - 自连接就是说，在同一个数据表中，看做是两个表，表示查找每个人的领导，如果没有领导，则显示无领导
        - 把一张表看做成两张表，一张员工表，一张领导表，都是emp表
        - select e.ename, el.ename from emp e left join emp el on e.mgr=el.empno;
- 自然连接：natural join (join) | natural left join(同 left join) | natural rignt join(同 rignt join)
    - 自然连接会自然判断，以两个表相同的字段为连接条件，然后返回查询结果
    - 注意：内连接不写连接条件会出现笛卡尔积的结果，应该避免这种情况，而外连接不谢连接条件会报错
        - select * from emp e natural join dept d ; 
        - select * from emp e natural left join dept d ; 
        - select * from emp e natural right join dept d ; 

### 子查询（ANY子查询，IN子查询，SOME子查询，ALL子查询）
- 子查询解决的问题
    - 谁的薪资比老王高？？？
    - select * from emp where sal > 2000;
    - select sal from emp where ename='老王';
    - 定义： 子查询允许把一个查询嵌套在另一个查询当中。具体操作如下：
    - select * from emp where sal > (select sal from emp where ename='老王');
    - 子查询又叫做内部查询，相当于内部查询。包含内部查询的就称为外部查询。子查询的结果会被主查询所使用
- 注意的问题：
    - 括号
    - 在主查询的 where select having from 后面，都可以使用子查询
    - 不可以在 group by 后面使用子查询
    - 主查询和子查询可以不是同一张表：只有子查询返回的值，主查询可以使用
        - 需求：查询部门名称是人力的员工信息
            - 第一种方式：利用子查询
                - select * from emp where deptno=(select deptno from dept where dname='人力部');
            - 第二种方式：利用关联查询
                - select * from emp e, dept d where e.deptno=d.deptno and d.dname='人力部';
        - 根据SQL优化，尽量使用多表查询
            - 绝大多数子查询在最终执行的时候，都是转换成一个多表查询来执行的。通过SQL执行计划可以看出来。
            - 通过SQL执行计划，会发现两种方式执行的是一样的。
    - from后面的子查询
        - 需求：查询员工号 姓名 月薪
            - select empno, ename, sal from emp;
            - select * from (select empno, ename, sal from emp) emp2;
    - 一般不在子查询中排序
    - 一般先执行子查询，再去执行主查询

#### ANY关键字
- 假设any内部的查询返回结果个数是三个，如：result1，result2，result3，那么
- select ... from ... where a > any (...);
- select ... from ... where a > result1 or a > result2 or a > result3;
- 需求：查询工资比1号部门中任意一个员工高的信息
    - select sal from emp where deptno=1;
    - select * from emp where sal > any(select sal from emp where deptno = 1);

#### ALL关键字
- all关键字与any关键字类似，只要把上面的or改成and， any改成all即可
- select ... from ... where a > all (...);
- select ... from ... where a > result1 and a > result2 and a > result3;
- 需求：查询工资比1号部门中所有员工高的信息
    - select sal from emp where deptno=1;
    - select * from emp where sal > all(select sal from emp where deptno = 1);

#### SOME关键字
- some关键字和any关键字是一样的功能，写法也一模一样
- select ... from ... where a > some (...);
- select ... from ... where a > result1 or a > result2 or a > result3;
- 需求：查询工资比1号部门中任意一个员工高的信息
    - select sal from emp where deptno=1;
    - select * from emp where sal > some(select sal from emp where deptno = 1);

#### IN关键字
- in运算符用于where表达式中，以列表向的形式支持多个选择。语法如下：
    - where column in (v1, v2, v3....);
    - where column not in (v1, v2, v3....);
    - 当in前面加上运算符的时候，表示与in相反的意思，即不在这些列表项中选择
    - select * from emp where in (1,2);
    - select * from emp where deptno not in (1,2);
- 案例：查询部门名称是人力和销售的员工 
    - select * from emp where deptno in (select deptno from dept where dname='人力部' or dname='销售部');
- 删除数据也可以使用in
    - delete from emp where deptno in (1,3);